<a href="https://colab.research.google.com/github/ShesterG/Twitter-Sentiment-Analysis/blob/master/notebooks/12_smv_model_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SVM with Smote Model for Twitter Sentiment Analysis

This model will use SVM with tf/idf to classify the tweets.

It will reduce the dimensionality of the feature set by adjusting the minimum document frequency of words, in an attempt to get better results.

It will use SMOTE to synthesize some data to balance the classes.

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Imports and Constants

In [2]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, plot_confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.preprocessing import LabelBinarizer
#import evaluation_functions
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import ast


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
DATA_FILE_PATH = '/content/drive/MyDrive/NLPGh/'
CLEAN_DATA_FILE_NAME = 'Book6Clean.csv'
TOKENIZED_DATA_FILE_NAME = 'Book6Tokenized.csv'
pd.set_option('display.max_colwidth', None)
RANDOM_STATE = 42

# Load Data

In [4]:
df = pd.read_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME)

In [5]:
df.head()

,Unnamed: 0,tweet,location_x,pretweet,candidate,LRPredScore,LRConfi,Sentiment,Unnamed: 8,tokens
0,0,@NsiaAvena @NAkufoAddo They don't know anything about peace,NaN,they dont know anyth about peac,Nana,NaN,NaN,-1,NaN,"['they', 'dont', 'know', 'anyth', 'about', 'peac']"
1,2,"@NAkufoAddo H. E Nana please follow up on this FAKE NEWS, LIES and sue BBC for peddling false information. Hold alÃ¢â¬Â¦ https://t.co/SdwnMsxeUV",NaN,nana pleas follow thi fake new lie and sue bbc for peddl fals inform hold al,Nana,-3.0,1.0,0,NaN,"['nana', 'pleas', 'follow', 'thi', 'fake', 'new', 'lie', 'and', 'sue', 'bbc', 'for', 'peddl', 'fals', 'inform', 'hold', 'al']"
2,4,@BarrowPresident @NAkufoAddo good to go may God bless an protect you,NaN,good may god bless protect you,Nana,NaN,NaN,1,NaN,"['good', 'may', 'god', 'bless', 'protect', 'you']"
3,6,"@ChipsJhay @TruthFreema @AOuattara_PRCI @NAkufoAddo No, you haven't. He's using his freedom of speech as it is. He's covered.","Afamase Akotom, Ghana",no you havent he use hi freedom speech is he cover,Nana,NaN,NaN,0,NaN,"['no', 'you', 'havent', 'he', 'use', 'hi', 'freedom', 'speech', 'is', 'he', 'cover']"
4,7,@IssahGerard @HEgyiri @NAkufoAddo Please who is this and how is he your in law??,NaN,pleas who thi and how your law,Nana,-1.0,1.0,0,NaN,"['pleas', 'who', 'thi', 'and', 'how', 'your', 'law']"


In [6]:
df = df[df.Sentiment != 0]

In [7]:
# convert list of strings represented as a string to a list of strings
df.tokens = df.tokens.map(ast.literal_eval)

In [8]:
data = df[['tokens', 'Sentiment']]

In [9]:
data.head()

,tokens,Sentiment
0,"[they, dont, know, anyth, about, peac]",-1
2,"[good, may, god, bless, protect, you]",1
5,"[presid, receiv, the, grace, god, almighti, what, ha, ordain, you, for, jesu, mighti, name]",1
6,"[gad, cant, believ, itu, are, congratul, dem, npp, parti, are, corrupt]",-1
7,"[pleas, may, god, continu, strengthen, you, more, pleas]",1


# Split Data

In [10]:
"""
X_train, X_test, y_train, y_test = train_test_split(data['tokens'], 
                                                    data['Sentiment'],
                                                   test_size = 0.2,
                                                   random_state = RANDOM_STATE,
                                                   stratify = data['Sentiment'])
"""  

X = data['tokens']
y = data['Sentiment']

In [ ]:
#X_train.shape

In [ ]:
#X_test.shape

# Encode Labels

In [11]:
le = LabelEncoder()

In [ ]:
#y_train_enc = le.fit_transform(y_train)

In [ ]:
#y_test_enc = le.transform(y_test)

In [12]:
y_enc = le.fit_transform(y)

In [ ]:
kf = KFold(2000, n_folds=10)

accuracy = []
fold = 0

# Evaluate Adjustments to Minimum Document Frequency

In [13]:
def passthrough(doc):
    """passthrough function for use in the pipeline because the text is already tokenized"""
    return doc

In [26]:
def train_and_eval_model(min_df):
    """
    Train and Evaluate and Bag of Words Representation with a SVM
    classifier with a specified minimum document frequency.
    """
    
    pipeline = Pipeline([('bow',CountVectorizer(min_df=min_df, 
                                                preprocessor=passthrough, 
                                                tokenizer=passthrough)),
                         ('tfidf', TfidfTransformer()),
                         ('smote', SMOTE(random_state=RANDOM_STATE, 
                                         n_jobs=-1)),
                         ('SVM', SVC(kernel = 'linear', probability = True, random_state = RANDOM_STATE))
                         ])
    

    #scores = cross_validation.cross_val_score(Pipeline, X_train,y_train_enc, cv=10)
    #pipeline.score
    scoring = ['accuracy','f1','precision','recall']
    scores = cross_validate(pipeline, X, y, cv=10, scoring=scoring)
    sorted(scores.keys())


    #score_mean = scores.mean()


    #pipeline.fit(X_train,y_train_enc)
    
    print(f'MINIMUM DOCUMENT FREQUENCY = {min_df}')
    print('')
    
    """
    #y_pred = cross_val_predict(Pipeline, X, Y, cv=10)
    
    
    #y_pred = pipeline.predict(X_test)



    #conf_matrix = confusion_matrix(y_true=y_test_enc, y_pred=y_pred)
    #confusion_matrix(y_true=y_train_enc, y_pred=pipeline.predict(X_train),labels='Training')

    
    #classi_report = classification_report(y_true=y_test_enc, y_pred=y_pred, target_names=['Negative', 'Postive'])
    #print(classi_report)
    ['fit_time', 'score_time', 'test_prec_macro', 'test_rec_macro',
 'train_prec_macro', 'train_rec_macro']

    
    #train_accuracy = accuracy_score(y_train_enc, pipeline.predict(X_train))
    
    test_accuracy = accuracy_score(y_test_enc, pipeline.predict(X_test))
    
    ave_macro_recall = recall_score(y_test_enc, pipeline.predict(X_test), average='macro')
    recall_scores = recall_score(y_test_enc, pipeline.predict(X_test), average=None)
    
    test_results = np.append(np.array([min_df, train_accuracy, test_accuracy, ave_macro_recall]),
                             recall_scores, score_mean)
    """
    a = scores['test_accuracy'].mean()
    f = scores['test_f1'].mean()
    p = scores['test_precision'].mean()
    r = scores['test_recall'].mean()
    test_results = np.append(np.array([min_df, a, f, p]),r)
    
    
   
    return test_results
    

In [27]:
metrics_summary = []

for min_df in range(1,11):
    test_scores = train_and_eval_model(min_df)
    metrics_summary.append(test_scores)

MINIMUM DOCUMENT FREQUENCY = 1

MINIMUM DOCUMENT FREQUENCY = 2

MINIMUM DOCUMENT FREQUENCY = 3

MINIMUM DOCUMENT FREQUENCY = 4

MINIMUM DOCUMENT FREQUENCY = 5

MINIMUM DOCUMENT FREQUENCY = 6

MINIMUM DOCUMENT FREQUENCY = 7

MINIMUM DOCUMENT FREQUENCY = 8

MINIMUM DOCUMENT FREQUENCY = 9

MINIMUM DOCUMENT FREQUENCY = 10



In [28]:
metrics_summary_df = pd.DataFrame(metrics_summary, 
                                  columns=['min doc freq',
                                           'accuracy',
                                           'f1',
                                           'precision',
                                          'recall'
                                          ])

metrics_summary_df

,min doc freq,accuracy,f1,precision,recall
0,1.0,0.898,0.896703,0.920915,0.880
1,2.0,0.885,0.882302,0.912120,0.862
2,3.0,0.882,0.878941,0.907666,0.860
3,4.0,0.876,0.873796,0.899135,0.858
4,5.0,0.876,0.873644,0.899578,0.856
5,6.0,0.874,0.870894,0.900009,0.850
6,7.0,0.875,0.870304,0.900599,0.850
7,8.0,0.878,0.873350,0.906147,0.852
8,9.0,0.873,0.870201,0.899427,0.852
9,10.0,0.870,0.864790,0.904170,0.840


In [ ]:
_,conf_matrix, y_pred = train_and_eval_model(min_df=2)
labels = ['Negative', 'Postive']

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
df2 = pd.DataFrame()

df2['X_test'] =  X_test
df2['y_test_enc'] =  y_test
df2['y_pred'] =  y_pred-1

df2[df2["y_test_enc"]==-1 & df2["y_pred"]==1].item()

In [ ]:
df2[np.logical_and.reduce([df2['y_test_enc'] == -1, df2['y_pred'] == 0])]

In [ ]:
df.loc[852,'tweet']

In [ ]:
df2[df2["y_test_enc"]==-1 & df2["y_pred"]==1].item()

# Summary

The addition of SMOTE oversampling generally improved the SVM as negative classes have better scores. 

In [ ]:
easy.py